# Bungee jump problem -  Root finding example

The analytical solution for the Bungee jump problem is:

$v(t) = \sqrt{\frac{gm}{C_d}} \tanh \left( \sqrt{\frac{gC_d}{m}} t \right)$

where $v(t)$ is velocity as a function of time $t$, $C_d$ is drag coefficient, $m$ is mass, $g$ is gravitational acceleration.

Question: given $g = 9.81$ m/s$^2$, $C_d = 0.25$ kg/m. At $t = 4$ s, velocity $v = 36$ m/s. Find $m$.

Solution: We can convert this to a root finding problem $f(x) = 0$:

$f(m) = \sqrt{\frac{gm}{C_d}} \tanh \left( \sqrt{\frac{gC_d}{m}} t \right) - v = 0$

In [41]:
g = 9.81  
Cd = 0.25 
t = 4
v = 36

## Bisection Method

You can find the details of Bisection method here: https://pythonnumericalmethods.berkeley.edu/notebooks/chapter19.03-Bisection-Method.html

In [42]:
import numpy as np

f = lambda m: (g*m/Cd)**0.5 * np.tanh((g*Cd/m)**0.5 * t) - v

## (1) Hand calucation

Step 1: Guess two values that give one negative and one positive solution, say, $m_l$ = 100, and $m_u$ = 200

In [43]:
ml, mu = 100, 200
print('f(ml) =',f(ml))
print('f(mu) =',f(mu))

f(ml) = -1.1973800012119113
f(mu) = 0.8602906131757351


Step 2: Now, we update the root as the average of these two values ($m_{mid}$) and check the function $f(m_{mid})$

In [44]:
m_mid = (ml + mu )/2
print(f(m_mid))

0.14204303065125146


Because $f(m_{mid})$ is positve, we can use m_mid to replace $m_r$ and repeat step 2

In [45]:
mu = m_mid
m_mid = (ml + mu )/2
print(f(m_mid))

-0.40860146335501213


Now, the new $f(m_{mid})$ is negative, we will use $m_mid$ to replace $m_l$ in the next iteration

In [46]:
ml = m_mid
m_mid = (ml + mu )/2
print(f(m_mid))

-0.11080406285225308


Do this again. 

In [47]:
ml = m_mid
m_mid = (ml + mu )/2
print('m is', m_mid, 'kg; f(m) is', f(m_mid))

m is 143.75 kg; f(m) is 0.02057713074902523


Now, f(m_mid) is quite small, and we can stop here for a hand calculation. For computer to do the same job, we need to set an accepted error to stop the iterations. 

## (2) Use a function

Let's use a function to process this for us. In the function, we need four input parameters: (1) the root-finding function, (2) left point, (3) right point, and (4) a tolerance for error 

In [48]:
def my_bisection(f, a, b, tol): 
    # approximates a root, R, of f bounded 
    # by a and b to within tolerance 
    # | f(m) | < tol with m the midpoint 
    # between a and b Recursive implementation
    
    # check if a and b bound a root
    if np.sign(f(a)) == np.sign(f(b)):
        raise Exception(
         "The scalars a and b do not bound a root")
        
    # get midpoint
    m = (a + b)/2
    
    if np.abs(f(m)) < tol:
        # stopping condition, report m as root
        return m
    elif np.sign(f(a)) == np.sign(f(m)):
        # case where m is an improvement on a. 
        # Make recursive call with a = m
        return my_bisection(f, m, b, tol)
    elif np.sign(f(b)) == np.sign(f(m)):
        # case where m is an improvement on b. 
        # Make recursive call with b = m
        return my_bisection(f, a, m, tol)

We can use the same initial guess and obtain the root using above function.

In [38]:
m = my_bisection(f, 100, 200, 0.01)
print('m = ', m, 'kg; f(m) = ', f(m))

m =  142.96875 kg; f(m) =  0.00472076784640052


Using bisection method, it is critical to choose correct inital guess values. For instace, guessing 50 and 100 will not return a good calcuation.

In [40]:
my_bisection(f, 50, 100, 0.01)

Exception: The scalars a and b do not bound a root

## False-position method

Bisection method is a brute force method and makes no use of information about the function itself. False-position method is a different method to find the new guess that uses the function and may converge more quickly than bisection method. Here is explaination of the false-position method: https://mathworld.wolfram.com/MethodofFalsePosition.html 

Keep in mind: in this method, the only difference is the equation to calucate new root:

In stead of using $x_r = (x_l + x_u)/2$, the new equation is used:

$x_r = x_u - \frac{f(x_u)(x_l-x_u)}{f(x_l)-f(x_u)}$

In [86]:
def falsi(f, a, b, tol, iter_max):
    if f(a) * f(b) > 0:
        raise Exception(
         "The scalars a and b do not bound a root")

    for i in range(iter_max):  
        c = b - f(b)*(a-b)/(f(a)-f(b))
        if i>0:           
            error = np.abs(c-c_old)
            if error < tol:
                print('solution converged')
                break
                       
        if f(c) == 0.0:
            print('exact solution found')
            break
        if f(a) * f(c) < 0:
            b = c
        else:
            a = c
        
        c_old = c
        
        if i == iter_max - 1:
            print('solution not converged, maximal iteration finished')
              
    return c

In [88]:
falsi(f, 100, 200, 0.01, 100)

solution converged


142.73924463697608